In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import numpy as np

In [13]:
base_url = "https://www.flipkart.com/samsung-galaxy-book-2-intel-core-i5-12th-gen-1235u-8-gb-512-gb-ssd-windows-11-home-np550-thin-light-laptop/product-reviews/itmcb89499a3c4e4?pid=COMGH3V3UUGM46BQ&lid=LSTCOMGH3V3UUGM46BQZAGUN0&marketplace=FLIPKART?page="

In [14]:
resData = urlopen(base_url)
htmlData = resData.read()
soup = bs(htmlData, 'html.parser')
pagination_div = soup.find('div', {"class":'_2MImiq _1Qnn1K'})
total_pages = int(pagination_div.span.text.split()[-1])
total_pages

25

In [21]:
def get_reviews_of_page(i):
    resData = urlopen(base_url+str(i))
    htmlData = resData.read()
    soup = bs(htmlData, 'html.parser')
    main_review_divs = soup.findAll('div', {"class":"col _2wzgFH K0kLPL"})

    cur_page_reviews = []
    for main_review_div in main_review_divs:
        review_head_div = main_review_div.find('p', {"class":"_2-N8zT"})
        review_heading = review_head_div.text
        
        review_body_div = main_review_div.find('div', {"class":'t-ZTKy'})
        review_body = review_body_div.text
        
        rating_div = main_review_div.find('div', {"class":"_3LWZlK _1BLPMq"})
        rating = rating_div.text.replace('READ MORE', '').strip()
        
        review_up_votes_div = main_review_div.find('div', {"class":'_1LmwT9'})
        review_up_votes = review_up_votes_div.text
        
        review_down_votes_div = main_review_div.find('div', {"class":'_1LmwT9 pkR4jH'})
        review_down_votes = review_down_votes_div.text
        
        cur_page_reviews.append({
            'heading':review_heading,
            'review':review_body,
            'rating':rating,
            'up_votes':review_up_votes,
            'down_votes':review_down_votes
        })
    print("cur_page_reviews:",cur_page_reviews)
    return cur_page_reviews

In [ ]:
final_review_array = []
for i in range(1, total_pages+1): #total_pages+1
    # get_reviews_of_page(i)
    final_review_array = final_review_array + get_reviews_of_page(i)


## Storing scraped data in csv file

In [31]:
import csv

In [ ]:
fileName = "flipkart-product-reviews.csv"

with open(fileName, 'w', encoding='utf-8', newline='') as file:
    headers = final_review_array[0].keys()
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    
    for review in final_review_array:
        writer.writerow(review)